<a href="https://colab.research.google.com/github/maherysm/Translation/blob/main/Translation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install datasets
!pip install evaluate


import torch
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM, Trainer, TrainingArguments, DataCollatorForSeq2Seq, MBartForConditionalGeneration, MBart50TokenizerFast
from datasets import load_dataset
import evaluate

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 14.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 7.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 8.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.0/40.0 MB 17.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 8.3 MB/s eta 0:00:00
  Attempting uninstall: pyarrow
    Found existing installation: pyarrow 14.0.2
    Uninstalling pyarrow-14.0.2:
      Successfully uninstalled pyarrow-14.0.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
cudf-cu12 24.4.1 requires pyarrow<15.0.0a0,>=14.0.1, but you have pyarrow 18.0.0 which is incompatible.
ibis-framework 8.0.0 requires pyarrow<16,>=2, but you have pyarrow 18.0.0 which is incompatible.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.0/84.0 kB 3.7 MB/s

# Hardware Accelerator

In [2]:
if torch.cuda.is_available():
    device = torch.device("cuda")
    print("on GPU.")
else:
    device = torch.device("cpu")
    print("on CPU.")

on GPU.


# Model and Tokenizer

In [6]:
# model_name = "Helsinki-NLP/opus-mt-en-fr"
model_name = "facebook/mbart-large-50"
model = MBartForConditionalGeneration.from_pretrained(model_name)
tokenizer = MBart50TokenizerFast.from_pretrained(model_name, src_lang="en_XX", tgt_lang="fr_XX")


# Dataset

In [7]:
dataset = load_dataset("wmt15", "fr-en")
train_data = dataset['train']
validation_data = dataset['validation']

README.md:   0%|          | 0.00/10.4k [00:00<?, ?B/s]

Resolving data files:   0%|          | 0/30 [00:00<?, ?it/s]

train-00000-of-00030.parquet:   0%|          | 0.00/253M [00:00<?, ?B/s]

train-00001-of-00030.parquet:   0%|          | 0.00/241M [00:00<?, ?B/s]

train-00002-of-00030.parquet:   0%|          | 0.00/243M [00:00<?, ?B/s]

...

test-00000-of-00001.parquet:   0%|          | 0.00/203k [00:00<?, ?B/s]

Generating train split:   0%|          | 0/40853137 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/4503 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/1500 [00:00<?, ? examples/s]

Loading dataset shards:   0%|          | 0/30 [00:00<?, ?it/s]

# Preprocess and Tokenize

In [8]:
def preprocess(sentences):
    inputs = [sentence['en'] for sentence in sentences["translation"]]
    targets = [sentence['fr'] for sentence in sentences["translation"]]

    model_inputs = tokenizer(inputs, max_length=512, truncation=True, padding="max_length")
    labels = tokenizer(text_target=targets, max_length=512, truncation=True, padding="max_length").input_ids

    model_inputs["labels"] = labels
    return model_inputs


train_data = train_data.map(preprocess, batched=True)
validation_data = validation_data.map(preprocess, batched=True)


Map:   0%|          | 0/40853137 [00:00<?, ? examples/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:4126: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your labels by using the argument `text_target` of the regular `__call__` method (either in the same call as your input texts if you use the same keyword arguments, or in a separate call.
  warnings.warn(


Map:   0%|          | 0/4503 [00:00<?, ? examples/s]

# Fine Tuning

In [ ]:
# dynamically pad sequences in batch to the length of the longest sequence in that batch
# collator = DataCollatorForSeq2Seq(tokenizer=tokenizer, model=model, padding=True)

training_args = TrainingArguments(
    output_dir="./results",
    eval_strategy="epoch",
    per_device_train_batch_size=4,
    per_device_eval_batch_size=4,
    learning_rate=5e-5,
    weight_decay=0.01,
    num_train_epochs=1,
    save_steps=10_000,
    save_total_limit=2,
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_data,
    eval_dataset=validation_data,
    # data_collator=collator
)

trainer.train()
